In [44]:
import us

# 9, 8, 7, 2, 3, 5

In [45]:
df = pd.read_csv('education.csv')

In [46]:
# 2 Have district results on my own, skipping
# pd.read_csv('data/2_Puhalendran_Seltzer_district_result.docx', encoding='latin-1')

# 3 I have no idea why first name popularity is relevant but ok
# df3 = pd.read_csv('data/3_Sundar_Solai_Susan_Xu_feature.csv', encoding='latin-1')
# df3['state'] = df3['district'].apply(lambda x : x.split(' ')[0].lower().capitalize())
# df3['area'] = 

In [47]:
# 5 Race
df5 = pd.read_csv('data/5_Miruna_Everett_Peter_Minority_Feature.csv', encoding='latin-1').iloc[:,1:]

In [48]:
missing_race = pd.read_csv('ACS_11_5YR_S0501/ACS_11_5YR_S0501.csv')[['GEO.display-label', 'HC01_EST_VC20']]
missing_race.columns = ['congressional_district', 'minority_percentage']
missing_race['minority_percentage'] = 1 - missing_race['minority_percentage']/100

In [49]:
df5 = pd.concat([df5, missing_race], axis=0, sort=False)

In [50]:
df5 = (df5.merge(df[['name', 'state_fips', 'district', 'congress_session']], 
                left_on='congressional_district', right_on='name', how='left')
       .drop(['congressional_district', 'candidates','year'], axis=1))

In [51]:
years = {'106th Congress': 1998, 
 '109th Congress': 2004, 
 '110th Congress': 2006, 
 '111th Congress': 2008,
 '112th Congress': 2010, 
 '113th Congress': 2012, 
 '114th Congress': 2014, 
 '115th Congress': 2016}
df5['raceyear'] = df5['congress_session'].apply(lambda x: years[x])

In [52]:
# 7 
df7 = pd.read_csv('data/7_presidential approval ratings  Wanqian, Russell, Eric.csv')
df7['state_fips'] = df7['district'].apply(lambda x : us.states.lookup(x.split('-')[0]).fips)
df7['district'] = df7['district'].apply(lambda x : x.split('-')[1].zfill(2))
df7['lastname'] = df7['name'].apply(lambda x : x.split(', ')[0])
df7.columns = ['raceyear', 'cand_name', 'district', 'pres_approval', 'same_as_pres_party', 'state_fips', 'lastname']
df7['district'] = df7['district'].astype(int)
df7['state_fips'] = df7['state_fips'].astype(int)
df7.loc[df7['raceyear'] == 2016, 'lastname'] = df7.loc[df7['raceyear'] == 2016, 'cand_name'].apply(lambda x : x.split(' ')[-1])

In [53]:
reps = df[['state_fips', 'district', 'raceyear', 'repcandidate']].copy()
reps['lastname'] = reps['repcandidate'].apply(lambda x: x.split(', ')[0] if type(x) is str else None)
reps = reps.merge(df7, on=['raceyear', 'district', 'state_fips', 'lastname'], how='left')

In [54]:
dems = df[['state_fips', 'district', 'raceyear', 'demcandidate']].copy()
dems['lastname'] = dems['demcandidate'].apply(lambda x: x.split(', ')[0] if type(x) is str else None)
dems = dems.merge(df7, on=['raceyear', 'district', 'state_fips', 'lastname'], how='left')

In [55]:
df7 = (reps.drop(['lastname', 'cand_name'], axis=1)
 .merge(dems.drop(['lastname', 'pres_approval', 'cand_name'], axis=1), 
        on=['state_fips', 'district', 'raceyear'], suffixes=('_rep', '_dem'))).sort_values('raceyear')

In [56]:
# 8: weird
df8 = pd.read_csv('data/8_Daniel Qu, Theodore Liu, and Jason Ren.csv')
df8.columns = ['raceyear', 'state', 'district', 'type']
df8['incumbent_party'] = (df8['type'].apply(lambda x : "R" 
                          if x in [0,1,2] else "D" if x in [3,4,5] else "I"))
df8 = df8[['raceyear', 'state', 'district', 'incumbent_party']].copy()

In [57]:
# 9
df9 = pd.read_csv('data/9_Gloria, Nick, Abhi.csv', sep='\t', header=None).iloc[:,:5]
df9.columns = ['raceyear', 'state', 'district', 'candidate', 'med_income']
df9 = df9[['raceyear','state','district','med_income']].drop_duplicates()
df9['log_med_income'] = np.log(df9['med_income'])

## Main

In [193]:
df = pd.read_feather('past_house.feather')

/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [194]:
# Only unopposed
df = df[df['repcandidate'].notnull() & df['demcandidate'].notnull()]

In [195]:
def clean_district(x):
    lst = x.split(' ')
    t = lst[1:]
    if len(t) == 1: 
        try:
            return str(int(t[0])).zfill(2)
        except:
            return '00'

In [196]:
df['district'] = df['area'].apply(clean_district)

In [197]:
cols = ['state', 'raceyear', 'district', 'repcandidate', 'repstatus', 
        'demcandidate', 'demstatus', 'pluralityparty', 'repvotesmajorpercent',
        'demvotesmajorpercent']

In [198]:
df = df[cols].copy()

In [199]:
df['rep_score'] = df['repvotesmajorpercent'] / (df['repvotesmajorpercent'] + df['demvotesmajorpercent'])

In [200]:
df['rep_incumbent'] = (df['repstatus'] == 'Incumbent')
df['dem_incumbent'] = (df['demstatus'] == 'Incumbent')

In [201]:
df = df.drop(['repstatus', 'demstatus', 'pluralityparty', 
         'repvotesmajorpercent', 'demvotesmajorpercent'], 
        axis=1)

In [245]:
df['state_fips'] = df['state'].apply(lambda x: int(us.states.lookup(x).fips))

In [247]:
df.reset_index(drop=True).to_feather('data/main.feather')

# Merge

In [59]:
df = pd.read_feather('data/main.feather')

df5.head()

df5 = df5.drop(['name','congress_session'], axis=1)
df5['district'] = df5['district'].astype(str).str.zfill(2)

df7['district'] = df7['district'].astype(str).str.zfill(2)

df9['district'] = df9['district'].astype(str).str.zfill(2)

education = pd.read_csv('education.csv')

education = education[['percent_bachelor_or_above', 'state_fips', 'district', 'raceyear']].drop_duplicates()

education['district'] = education['district'].astype(str).str.zfill(2)

education['percent_bachelor_or_above'] /= 100

(df.merge(df5.drop_duplicates(), how='left')
 .merge(df7.drop_duplicates(), how='left')
 .merge(df9.drop_duplicates().drop('med_income', axis=1), how='left')
 .merge(education, how='left')
 .sort_values('raceyear', ascending=False)
 .reset_index(drop=True)).to_feather('data/cleaned_train_Kevin.feather')

/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [324]:
test = pd.read_feather('data/main_2018.feather')

/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [325]:
test['state_fips'] = test['state_fips'].astype(int)

In [326]:
test['district'] = test['district'].astype(str).str.zfill(2)

In [327]:
df9 = df9.query('state != "U.S. Virgin Islands"').copy()
df9['state_fips'] = df9['state'].apply(lambda x : int(us.states.lookup(x).fips))

In [344]:
test = (test.merge(df5.query('raceyear==2016')
            .drop('raceyear', axis=1)
            .drop_duplicates(), 
           how='left')
 .merge(df9.query('raceyear==2016').drop(['med_income', 'state','raceyear'], axis=1)
        .drop_duplicates(),
        how='left')
 .merge(education.query('raceyear==2016').drop(['raceyear'], axis=1), how='left'))

approv = pd.read_csv('data/current_approv.csv', header=None).iloc[:,[0,6]]

approv.columns = ['s', 'pres_approval']

approv['pres_approval'] = approv['pres_approval'].apply(lambda x: float(x[:-1])/100)

approv['state_fips'] = approv['s'].apply(lambda s: int(us.states.lookup(s).fips))

test['same_as_pres_party_rep'] = True
test['same_as_pres_party_dem'] = False

In [345]:
test.merge(approv.drop('s', axis=1), how='left').to_feather('data/cleaned_test_Kevin.feather')

In [60]:
test = pd.read_feather('data/cleaned_test_Kevin.feather')
train = pd.read_feather('data/cleaned_train_Kevin.feather')

/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [66]:
train.head().T

,0,1,2,3,4
state,Alabama,Kansas,California,California,California
raceyear,2016.0000,2016.0000,2016.0000,2016.0000,2016.0000
district,02,04,05,04,03
repcandidate,"Roby, Martha","Pompeo, Mike","Santamaria, Carlos","McClintock, Tom","Cleek, N. Eugene"
demcandidate,"Mathis, Nathan","Giroux, Daniel B.","Thompson, Mike","Derlet, Robert W.","Garamendi, John"
rep_score,0.5462,0.6720,0.2313,0.6272,0.4065
rep_incumbent,True,True,False,True,False
dem_incumbent,False,False,True,False,True
state_fips,1,20,6,6,6
minority_percentage,0.3698,0.1753,0.3785,0.1561,0.3347


In [74]:
test['raceyear'] = 2018
test['state'] = test['state_fips'].apply(lambda x : us.states.lookup(str(x).zfill(2)).name)

In [79]:
pd.concat([train, test], sort=False).reset_index(drop=True).to_feather('data/kevin_joint.feather')